**SaleId** : Satışid
**SaleDate** : SatışTarihi
**Price** : Satışiçin ödenen fiyat
**ConceptName**: Otel konsept bilgisi
**SaleCityName**: Otelin bulunduğu şehir bilgisi
**CheckInDate** : Müşterinin otele girişi tarihi
**CInDay**: Müşterinin otele girişgünü
**SaleCheckInDayDiff**: Check in ile giriştarihi gün farkı
**Season**: Otele giriştarihindeki sezon bilgisi


# Görev 2: SaleCheckInDayDiff değişkenini kategorik bir değişkene çeviriniz.

- SaleCheckInDayDiff değişkeni müşterinin CheckIn tarihinden ne kadar önce satin alımını tamamladığını gösterir.
- Aralıkları ikna edici şekilde oluşturunuz. Örneğin: ‘0_7’, ‘7_30’, ‘30_90’, ‘90_max’ aralıklarını kullanabilirsiniz.
- Bu aralıklar için "Last Minuters", "Potential Planners", "Planners", "Early Bookers“ isimlerini kullanabilirsiniz.

In [105]:
import pandas as pd
import numpy as np
df = pd.read_excel("miuul_gezinomi.xlsx")

In [104]:
bins =[-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Brookers"]

In [106]:
df["EB_SCORE"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels=labels)
df.head(50).to_excel("eb_score.xlsx", index=False)

# Görev 3:  COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?

Şehir-Concept-EB Score, Şehir-Concept- Sezon, Şehir-Concept-CInDay kırılımında ortalama ödenen ücret ve yapılan işlem sayısı cinsinden
inceleyiniz ?

In [111]:
df.groupby(["SaleCityName", "ConceptName", "EB_SCORE"]).agg({"Price":["mean","count"]}).head(10)

Price       
                                                     mean  count
SaleCityName ConceptName    EB_SCORE                            
Antalya      Herşey Dahil   Last Minuters       62.752916  14148
                            Potential Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early Brookers      66.494885   3281
             Oda + Kahvaltı Last Minuters       65.353740    503
                            Potential Planners  57.735542     75
                            Planners            39.851330     15
                            Early Brookers      35.002594      5
             Yarım Pansiyon Last Minuters       70.433576    204
                            Potential Planners  55.639242     39

In [112]:
df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price":["mean","count"]}).head(10)

Price       
                                          mean  count
SaleCityName ConceptName    Seasons                  
Antalya      Herşey Dahil   High     64.920065  27126
                            Low      61.552427   3667
             Oda + Kahvaltı High     66.267969    303
                            Low      60.666730    295
             Yarım Pansiyon High     73.259029    118
                            Low      62.001678    138
Aydın        Herşey Dahil   High     54.946560  10103
                            Low      33.680699    473
             Oda + Kahvaltı High     30.387530     27
                            Low      44.451874     11

In [113]:
df.groupby(["SaleCityName", "ConceptName", "CInDay"]).agg({"Price":["mean","count"]}).head(10)

Price      
                                            mean count
SaleCityName ConceptName    CInDay                    
Antalya      Herşey Dahil   Friday     62.658940  4136
                            Monday     63.259143  6831
                            Saturday   64.415526  4741
                            Sunday     65.849177  3818
                            Thursday   62.892734  3898
                            Tuesday    66.771658  3760
                            Wednesday  67.173832  3609
             Oda + Kahvaltı Friday     63.128165   114
                            Monday     57.172843    66
                            Saturday   58.012419   117

# Görev 4:  City-Concept-Season kırılımının çıktısını PRICE'a göre sıralayınız.

Elde ettiğiniz çıktıyı agg_df olarak kaydediniz.

In [114]:
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price":"mean"}).sort_values("Price", ascending=False)

In [115]:
agg_df.head()

Price
SaleCityName ConceptName    Seasons            
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
İzmir        Yarım Pansiyon High      87.657303
Diğer        Herşey Dahil   Low       87.310882
                            High      83.787273

# Görev 5:  Indekste yer alan isimleri değişken ismine çeviriniz.

Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir. Bu isimleri değişken isimlerine çeviriniz.

In [116]:
agg_df.reset_index(inplace=True)

In [117]:
agg_df.head(12)

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935448
1,Girne,Herşey Dahil,Low,90.935944
2,İzmir,Yarım Pansiyon,High,87.657303
3,Diğer,Herşey Dahil,Low,87.310882
4,Diğer,Herşey Dahil,High,83.787273
5,İzmir,Herşey Dahil,High,74.748363
6,İzmir,Herşey Dahil,Low,74.308287
7,Antalya,Yarım Pansiyon,High,73.259029
8,Antalya,Oda + Kahvaltı,High,66.267969
9,Antalya,Herşey Dahil,High,64.920065


# Görev 6:  Yeni seviye tabanlı müşterileri (persona) tanımlayınız.

- Yeni seviye tabanlı satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.
- Yeni eklenecek değişkenin adı: **sales_level_based**
- Önceki soruda elde edeceğiniz çıktıdaki gözlemleri bir araya getirerek **sales_level_based** değişkenini oluşturmanız gerekmektedir.

In [118]:
agg_df["sales_level_based"] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: "_".join(x).upper(), axis=1)

In [119]:
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH


# Görev 7:  Yeni müşterileri (personaları) segmentlere ayırınız.

- Yeni personaları PRICE’a göre 4 segmente ayırınız.
- Segmentleri SEGMENT isimlendirmesi ile değişken olarak agg_df’e ekleyiniz.
- Segmentleri betimleyiniz (Segmentlere göre group by yapıp price mean, max, sum’larını alınız).

In [120]:
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])

In [121]:
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH,A
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW,A
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH,A
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW,A
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH,A


In [122]:
agg_df.groupby("SEGMENT").agg({"Price":["mean","max","sum"]})

Price                        
              mean         max         sum
SEGMENT                                   
D        33.366288   39.479630  300.296592
C        44.887654   54.141797  403.988887
B        60.274498   64.920065  542.470478
A        82.467833  103.935448  742.210498

# Görev 8:  Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini  tahmin ediniz.

1. Antalya’da herşey dahil ve yüksek sezonda tatil yapmak isteyen bir kişinin ortalama ne kadar gelir kazandırması beklenir?
2. Girne’de yarım pansiyon bir otele düşük sezonda giden bir tatilci hangi segmentte yer alacaktır?

In [123]:
agg_df.sort_values(by="Price")

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
35,Aydın,Yarım Pansiyon,Low,25.271799,AYDIN_YARIM PANSIYON_LOW,D
34,Aydın,Oda + Kahvaltı,High,30.387530,AYDIN_ODA + KAHVALTI_HIGH,D
33,Aydın,Yarım Pansiyon,High,32.626152,AYDIN_YARIM PANSIYON_HIGH,D
32,Muğla,Yarım Pansiyon,Low,32.681812,MUĞLA_YARIM PANSIYON_LOW,D
31,İzmir,Oda + Kahvaltı,Low,33.555235,İZMIR_ODA + KAHVALTI_LOW,D
30,Aydın,Herşey Dahil,Low,33.680699,AYDIN_HERŞEY DAHIL_LOW,D
29,Diğer,Oda + Kahvaltı,High,34.839543,DIĞER_ODA + KAHVALTI_HIGH,D
28,Muğla,Oda + Kahvaltı,Low,37.774192,MUĞLA_ODA + KAHVALTI_LOW,D
27,Girne,Oda + Kahvaltı,High,39.479630,GIRNE_ODA + KAHVALTI_HIGH,D
26,Diğer,Yarım Pansiyon,High,39.731916,DIĞER_YARIM PANSIYON_HIGH,C


In [124]:
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"

In [125]:
agg_df[agg_df["sales_level_based"]==new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH,B
